<a href="https://colab.research.google.com/github/VGODIE/ML_kaggle_competitions/blob/master/Taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns

In [0]:
! pip3 install torch torchvision

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vgodie","key":"5f216b50c6912d8ac1efdc0406da9f50"}'}

In [0]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
! kaggle competitions download -c burned-taxi-prediction

 45% 9.00M/20.2M [00:00<00:00, 18.8MB/s]
100% 20.2M/20.2M [00:00<00:00, 34.4MB/s]
100% 4.49M/4.49M [00:00<00:00, 31.3MB/s]

  0% 0.00/8.06k [00:00<?, ?B/s]
100% 8.06k/8.06k [00:00<00:00, 8.22MB/s]


In [0]:
! unzip train.csv.zip
! unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [0]:
df = pd.read_csv("train.csv")

In [0]:
df.head()

,cancel_time,driver_found,due,f_class,lat,lon,s_class,t_class,burned
0,55,False,2014-01-01 00:09:32.000,econom,55.750130,37.823242,NaN,NaN,True
1,-1,True,2014-01-01 00:09:32.000,econom,55.750130,37.823242,NaN,NaN,False
2,-1,True,2014-01-01 00:10:00.000,econom,55.651582,37.340891,NaN,NaN,False
3,-1,True,2014-01-01 00:10:00.000,econom,55.633404,37.797595,NaN,NaN,False
4,-1,True,2014-01-01 00:10:00.000,econom,55.770330,37.519917,NaN,NaN,False


In [0]:
len(df[df.cancel_time < 0].cancel_time) / len(df)

0.8362006896551725

# Massive data preprocessing

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [0]:
target = df.burned.values
lab_enc = LabelEncoder()
target = lab_enc.fit_transform(target)
target

array([1, 0, 0, ..., 0, 0, 1])

In [0]:
driver_found = df.driver_found.values
dr_enc = LabelEncoder()
driver_found = dr_enc.fit_transform(driver_found)
driver_found

array([0, 1, 1, ..., 1, 1, 1])

In [0]:
cancel_time = df.cancel_time.values

In [0]:
kr_coor = np.array([[55.751694], [37.617218]]).reshape(1, 2)

In [0]:
from sklearn.metrics.pairwise import pairwise_distances

In [0]:
coor = df[["lat", "lon"]].values

In [0]:
distances = pairwise_distances(X = coor, Y = kr_coor)

In [0]:
dom = np.array([[55.245800], [37.535300]]).reshape(1,2) #домодедово
sher = np.array([[55.981317], [37.410052]]).reshape(1,2) #шарик
vnuk = np.array([[55.364200], [37.174500]]).reshape(1,2) #внуково
zhuk = np.array([[55.561365], [38.117674]]).reshape(1,2) #жуковский

dist = pairwise_distances(X = coor, Y = np.concatenate((dom, sher, vnuk, zhuk), axis=0))
dist = np.min(dist, axis=1).reshape(-1, 1)

In [0]:
sq_dist = dist**2

Поймем где Москва, где нет

In [0]:
sq_distances = distances**2

In [0]:
boarder_coor = np.array([55.768301, 37.370926]).reshape(1, 2)

In [0]:
bound = pairwise_distances(kr_coor, boarder_coor)[0, 0]

In [0]:
moscow = []
for elem in distances:
  if elem > bound:
    moscow.append(0)
  else:
    moscow.append(1)
moscow = np.array(moscow).reshape(-1, 1)

class encoding

In [0]:
from sklearn.impute import SimpleImputer
sim = SimpleImputer(missing_values = np.nan, strategy = "constant")

In [0]:
f_class = df.f_class
f_class = sim.fit_transform(f_class.values.reshape(1, -1)).reshape(len(f_class), 1)
class_enc = LabelEncoder()
first = class_enc.fit_transform(f_class)
first = tf.keras.utils.to_categorical(first, num_classes=4)[:, :-1]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
s_class = sim.fit_transform(df.s_class.values.reshape(-1, 1)).reshape(len(df.s_class), -1)
second = class_enc.fit_transform(s_class)
second = tf.keras.utils.to_categorical(second, num_classes=4)[:, :-1]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
t_class = sim.fit_transform(df.t_class.values.reshape(1, -1)).reshape(len(df.t_class), 1)
third = class_enc.fit_transform(t_class)
third = tf.keras.utils.to_categorical(third, num_classes=4)[:, :-1]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Time processing

In [0]:
import datetime

In [0]:
date = list(df.due)

In [0]:
year = []
alles = []
times = []
for i in range(len(date)):
    year.append(date[i][0:4])
    alles.append(date[i][0:10])
    times.append(date[i][11:19])

In [0]:
months = []
days = []
years = []
for elem in alles:
    a = list(elem.split("-"))
    days.append(a[2])
    months.append(a[1])
    years.append(a[0])
day = list(map(int, days))
month = list(map(int, months))

In [0]:
week_day = []
for i in range(len(years)):
    week_day.append(datetime.datetime(int(years[i]), int(month[i]), int(day[i])).weekday())

In [0]:
holiday = []
for i in range(len(alles)):
    if (alles[i][5:7] == '01' and alles[i][8:10] == '01') or (alles[i][5:7] == '21' and alles[i][8:10] == '02') or (alles[i][5:7] == '23' and alles[i][8:10] == '02') or (alles[i][5:7] == '08' and alles[i][8:10] == '03') or (alles[i][5:7] == '22' and alles[i][8:10] == '02') or (alles[i][5:7] == '07' and alles[i][8:10] == '03') or (alles[i][5:7] == '09' and alles[i][8:10] == '03') or (alles[i][5:7] == '10' and alles[i][8:10] == '03') or (alles[i][5:7] == '02' and alles[i][8:10] == '01') or (alles[i][5:7] == '03' and alles[i][8:10] == '01') or (alles[i][5:7] == '04' and alles[i][8:10] == '01') or (alles[i][5:7] == '05' and alles[i][8:10] == '01') or (alles[i][5:7] == '06' and alles[i][8:10] == '01') or (alles[i][5:7] == '07' and alles[i][8:10] == '01') or (alles[i][5:7] == '08' and alles[i][8:10] == '01'):
        holiday.append(1) #праздничные
    else:
        holiday.append(0)

In [0]:
time_day = []
for i in range(len(times)):
    if times[i][0:2] == '23' or times[i][0:2] == '00' or times[i][0:2] == '01' or times[i][0:2] == '02' or times[i][0:2] == '03' or times[i][0:2] == '04' or times[i][0:2] == '05' or times[i][0:2] == '06':
        time_day.append(0) #ночь
    elif times[i][0:2] == '07' or times[i][0:2] == '08' or times[i][0:2] == '09' or times[i][0:2] == '10' or times[i][0:2] == '11' or times[i][0:2] == '12':
        time_day.append(1) #утро
    elif times[i][0:2] == '13' or times[i][0:2] == '14' or times[i][0:2] == '15' or times[i][0:2] == '16' or times[i][0:2] == '17':
        time_day.append(2) #день
    elif times[i][0:2] == '18' or times[i][0:2] == '19' or times[i][0:2] == '20' or times[i][0:2] == '21' or times[i][0:2] == '22':
        time_day.append(3) #вечер

In [0]:
picks = []
for i in range(len(times)):
    if times[i][0:2] == '08' or times[i][0:2] == '09' or times[i][0:2] == '18' or times[i][0:2] == '19':
        picks.append(1) #пик
    else:
        picks.append(0)

Preprocess cats

In [0]:
wd = tf.keras.utils.to_categorical(week_day, num_classes=7)[:, :-1]

In [0]:
picks = np.array(picks).reshape(-1, 1)

In [0]:
holiday = np.array(holiday).reshape(-1, 1)

In [0]:
time_day = tf.keras.utils.to_categorical(time_day, num_classes=4)[:, :-1]

Get all data together

In [0]:
lat, lon = df.lat.values.reshape(-1, 1), df.lon.values.reshape(-1, 1)

In [0]:
sq_lat = lat**2
sq_lon = lon**2

In [0]:
moscow = moscow.reshape(-1, 1)

In [0]:
holiday = holiday.reshape(-1,1)

In [0]:
num_feats = np.concatenate((lat, sq_lat, lon, sq_lon, distances, dist, sq_distances, sq_dist), axis=1)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(num_feats)
num_feats_scaled = sc.transform(num_feats)

In [0]:
X = np.concatenate((first, second, third, moscow, wd, picks, holiday, time_day, num_feats_scaled), axis=1)

In [0]:
second.shape

(1450000,)

In [0]:
Z = np.concatenate((lat, lon, distances, moscow), axis = 1)

# training



In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, target, random_state=21)

In [0]:
z_train, z_test, y_train, y_test = train_test_split(Z, target, random_state=42)

In [0]:
boost = GradientBoostingClassifier(learning_rate=0.05, n_estimators=150)

boost.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=150,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

model tuning

In [0]:
preds = boost.predict_proba(x_test)[:, 1]
boost.score(x_test, y_test)

0.8366317241379311

In [0]:
log = LogisticRegression(penalty="l2")
log.fit(x_train, y_train)
preds = log.predict_proba(x_test)

In [0]:
log.score(x_test, y_test)

0.8368358620689655

In [0]:
accuracy_score(y_test, preds)

In [0]:
roc_auc_score(y_test, preds)

0.5932879751472954

# test managing

In [0]:
test = pd.read_csv("test.csv")

In [0]:
date1 = list(test.due)

In [0]:
year1 = []
alles1 = []
times1 = []
for i in range(len(date1)):
    year1.append(date1[i][0:4])
    alles1.append(date1[i][0:10])
    times1.append(date1[i][11:19])

In [0]:
months1 = []
days1 = []
years1 = []
for elem in alles1:
    a1 = list(elem.split("-"))
    days1.append(a1[2])
    months1.append(a1[1])
    years1.append(a1[0])
day1 = list(map(int, days1))
month1 = list(map(int, months1))

In [0]:
week_day1 = []
for i in range(len(years1)):
    week_day1.append(datetime.datetime(int(years1[i]), int(month1[i]), int(day1[i])).weekday())

In [0]:
holiday1 = []
for i in range(len(alles1)):
    if (alles1[i][5:7] == '01' and alles1[i][8:10] == '01') or (alles1[i][5:7] == '21' and alles1[i][8:10] == '02') or (alles1[i][5:7] == '23' and alles1[i][8:10] == '02') or (alles1[i][5:7] == '08' and alles1[i][8:10] == '03') or (alles1[i][5:7] == '22' and alles1[i][8:10] == '02') or (alles1[i][5:7] == '07' and alles1[i][8:10] == '03') or (alles1[i][5:7] == '09' and alles1[i][8:10] == '03') or (alles1[i][5:7] == '10' and alles1[i][8:10] == '03') or (alles1[i][5:7] == '02' and alles1[i][8:10] == '01') or (alles1[i][5:7] == '03' and alles1[i][8:10] == '01') or (alles1[i][5:7] == '04' and alles1[i][8:10] == '01') or (alles1[i][5:7] == '05' and alles1[i][8:10] == '01') or (alles1[i][5:7] == '06' and alles1[i][8:10] == '01') or (alles1[i][5:7] == '07' and alles1[i][8:10] == '01') or (alles1[i][5:7] == '08' and alles1[i][8:10] == '01'):
        holiday1.append(1) #праздничные
    else:
        holiday1.append(0)

In [0]:
time_day1 = []
for i in range(len(times1)):
    if times1[i][0:2] == '23' or times1[i][0:2] == '00' or times1[i][0:2] == '01' or times1[i][0:2] == '02' or times1[i][0:2] == '03' or times1[i][0:2] == '04' or times1[i][0:2] == '05' or times1[i][0:2] == '06':
        time_day1.append(0) #ночь
    elif times1[i][0:2] == '07' or times1[i][0:2] == '08' or times1[i][0:2] == '09' or times1[i][0:2] == '10' or times1[i][0:2] == '11' or times1[i][0:2] == '12':
        time_day1.append(1) #утро
    elif times1[i][0:2] == '13' or times1[i][0:2] == '14' or times1[i][0:2] == '15' or times1[i][0:2] == '16' or times1[i][0:2] == '17':
        time_day1.append(2) #день
    elif times1[i][0:2] == '18' or times1[i][0:2] == '19' or times1[i][0:2] == '20' or times1[i][0:2] == '21' or times1[i][0:2] == '22':
        time_day1.append(3) #вечер

In [0]:
picks1 = []
for i in range(len(times1)):
    if times1[i][0:2] == '08' or times1[i][0:2] == '09' or times1[i][0:2] == '18' or times1[i][0:2] == '19':
        picks1.append(1) #пик
    else:
        picks1.append(0)

In [0]:
wd1 = tf.keras.utils.to_categorical(week_day1, num_classes=7)[:, :-1]

In [0]:
picks1 = np.array(picks1).reshape(-1, 1)

In [0]:
holiday1 = np.array(holiday1).reshape(-1,1)

In [0]:
time_day1 = tf.keras.utils.to_categorical(time_day1, num_classes=4)[:,:-1]

In [0]:
time_day1.shape

(343300, 3)

In [0]:
f_class_t = test.f_class
f_class_t = sim.fit_transform(test.f_class.values.reshape(1, -1)).reshape(len(f_class_t), 1)
t_first = class_enc.fit_transform(f_class_t)
t_first = tf.keras.utils.to_categorical(t_first, num_classes=4)[:,:-1]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
s_class_t = sim.fit_transform(test.s_class.values.reshape(-1, 1)).reshape(len(test.s_class.values), -1)
t_second = class_enc.fit_transform(s_class_t)
t_second = tf.keras.utils.to_categorical(t_second, num_classes=4)[:,:-1]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
t_class_t = sim.fit_transform(test.t_class.values.reshape(1, -1)).reshape(len(test.t_class), 1)
t_third = class_enc.fit_transform(t_class_t)
t_third = tf.keras.utils.to_categorical(t_third, num_classes=4)[:,:-1]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
lat_test, lon_test = test.lat.values.reshape(-1, 1), test.lon.values.reshape(-1, 1)

In [0]:
sq_lat_test = lat_test**2
sq_lon_test = lon_test**2

In [0]:
distances_test = pairwise_distances(X = test[["lat", "lon"]], Y = kr_coor)
sq_distances_test = distances_test**2

In [0]:
t_dist = pairwise_distances(X = test[["lat", "lon"]], Y = np.concatenate((dom, sher, vnuk, zhuk), axis=0))
t_dist = np.min(t_dist, axis=1).reshape(-1, 1)

In [0]:
sq_t_dist = t_dist**2

In [0]:
moscow_t = []
for elem in distances_test:
  if elem > bound:
    moscow_t.append(0)
  else:
    moscow_t.append(1)
moscow_t = np.array(moscow_t).reshape(-1, 1)

In [0]:
num_feats_test = np.concatenate((lat_test, sq_lat_test, lon_test, sq_lon_test, distances_test, t_dist, sq_distances_test, sq_t_dist), axis=1)


In [0]:
num_feats_test_sc = sc.fit_transform(num_feats_test)

In [0]:
X_test = np.concatenate((t_first, t_second, t_third, moscow_t, wd1, picks1, holiday1, time_day1,num_feats_test_sc), axis=1)

In [0]:
#Z_test = np.concatenate((lat_test, lon_test, distances_test, moscow_t), axis=1)

In [0]:
ans = boost.predict_proba(X_test)[:, 1]

In [0]:
final = pd.DataFrame(ans, columns=['Prob'])
final.to_csv('submit.csv', index_label='Id')